In [58]:
#Simple Bender's example


import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

#Objective: maximize income


#First stage: How much lumber to buy and how much work to hire

#Random variable: demand quantities

#second stage: How many desks, tables and chairs to produce (Quantity) <--- limitied by how much investments in stage 1

#SETUP
constants = {
            'scenarios':['High', 'Most likely', 'Low'],
            'probs': {'High':0.3, 'Most likely':0.4, 'Low':0.3}
            }

requirements = {
    "Desk": {
        "Lumber": 8,
        "Finishing": 4,
        "Carpentry": 2,
        
    },
    "Table": {
        "Lumber": 8,
        "Finishing": 4,
        "Carpentry": 2,

    },
    "Chair": {
        "Lumber": 6,
        "Finishing": 2,
        "Carpentry": 1.5,

    },
     "Cost": {
        "Lumber":  2,
        "Finishing": 4,
        "Carpentry": 5.2,

    }
}


In [59]:
import pyomo.environ as pyo
import numpy as np
from pyomo.environ import ConcreteModel,Set,RangeSet,Param,Suffix,Reals,NonNegativeReals,NonPositiveReals,Binary,Objective,minimize,maximize,value
from pyomo.core import Constraint,Var,Block,ConstraintList
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
import matplotlib.pyplot as plt
from calculations.datahandling import*
from calculations.data_processor import* 

## Initialization of data

In [60]:
#--Data handling--
#Read of parameters for portfolio
def InputData(data_file):
    inputdata = pd.read_excel(data_file)
    inputdata = inputdata.set_index('Parameter', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['hydro'] = inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Solar')
    data['solar']=inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Hydro1').drop('Hydro2')
    return data

data = InputData('data/Parameters.xlsx')


#Solar production based on forecast 
def read_solar_data(irrad_file, spec_file, start_date, end_date):
    input_data_PV = read_excel_data(spec_file)
    input_data_Irr = read_irr_data(irrad_file)
    PV_power = pv_power_estimated(input_data_PV,input_data_Irr)
    Solar_1=convert_to_dict(PV_power, start_date, end_date, 'H')
    Solar_p=scale_dict(Solar_1, 1)
    return Solar_p

#Start and end dates of the optimization
start_date='2018-05-28 00:00'
end_date='2018-05-28 23:00'     #Two time steps only

#Original forecast for solar power production
Solar_p=read_solar_data('data/Data_solar_irr_NOR.csv', 'data/PV_spec.xlsx', start_date, end_date)

S_high=scale_dict(Solar_p, 1.5)
S_avg=scale_dict(Solar_p, 1)
S_low=scale_dict(Solar_p, 0.5)

#Load 
L= {1:50, 3:20, 4:30, 5:50, 6:80, 7:50, 8:90, 9:110, 10:150, 11:120, 12:80, 13:70, 14:80, 15:90, 16:160, 17:170, 18:150, 19:120, 20:100, 21:70, 22:60, 23:50, 24:40} 

#--Constants--
Constants= {
    'Load_penalty':100, 
    'Hydro_cap':3000,
    'Scenarios':['S_high', 'S_avg', 'S_low'], 
    'probs':{'S_high':1/3, 'S_avg':1/3, 'S_low':1/3},
    'yi_s1':{'S_high':25, 'S_avg':25, 'S_low':25}, 
    'yi_s2':{'S_high':35, 'S_avg':35, 'S_low':35}     
}

# ----------------------Mathematical formulation ------------------------- #
# ----- First stage obejctive and constraints ------------------ #
def Obj_1st(model):
    return sum(model.yi[i] * model.p[i, j] for i in model.plants for j in model.periods) \
         + sum(model.ki[s] * model.phi[s, j] for s in model.solar for j in model.periods) \
         + sum(model.Li[n] * model.L_p[n, j] for n in model.penalty for j in model.periods) + model.alpha

def load_rule(model,j):
    return model.p['Hydro1',j] + model.p['Hydro2',j] + model.phi['Solar',j] + model.L_p['Load_penalty',j] == model.L[j]

def Hydro_capacity(model,i,j):
    return sum(model.p[i,j]) <= sum(model.H_cap[i,j])

def p_bounds(model,i,j):
    return (model.Pmin[i],model.Pmax[i])   

def Solar_rule(model,j):
    return model.phi['Solar',j] == Solar_p[j]

def CreateCuts(model,Ci):
    return(model.alpha <= model.Phi[Ci] + sum(model.Lambda[Ci,i]*(model.p[i]-model.x_hat[Ci,i]) for i in model.plants))  #only takes into account the hydro power production scheduling from H_cap

#def CreateCuts(model)
#    return(model.alpha >= model.Phi[])

#def CreateCuts(m,c):
#    return(m.alpha <= m.Phi[c] + sum(m.Lambda[c,i]*(m.x[i]-m.x_hat[c,i]) for i in m.I))  #only the variable x is taken into account as it is the only complicating variable
#the sensitivities are indicated by

# ----- Second stage ojbective and constraints --------------#
def Obj_2nd(model):
    return sum(model.probs[l] * (model.yi[i]*model.p[i,l,j] + model.ki[s] * model.phi[l, j] + model.Li[n] * model.L_p[l, j]) for i in model.plants for s in model.solar for l in model.scenarios for n in model.penalty for j in model.periods)

def load_rule_scenarios(model,s,j,l): #l in scenarios
    return model.p['Hydro1',l,j] + model.p['Hydro2',l,j] + (model.phi[s,l,j]) + model.L_p['Load_penalty'] == model.L[j]

def Hydro_capacity2(model, i,j,l):
    return model.p[i,j,l] <= model.H_cap[i,j,l]

def p_bounds(model,i,j):
    return (model.Pmin[i], model.Pmax[i])   

def Solar_high(model,j):
    return model.phi['S_high',j]==S_high[j]

def Solar_avg(model,j):
    return  model.phi['S_avg',j] == S_avg[j]

def Solar_low(model,j):
    return  model.phi['S_low',j] == S_low[j]

def rem_cap(Constants, X_hat,j):
    remaining_cap = 0
    return remaining_cap.append(Constants['Hydro_cap'] - sum(X_hat['Hydro1',j] + X_hat['Hydro2',j]))

## Model setup

In [61]:
# ------------------------- MODEL SETUP -------------------------- #

# Set up model 1st stage
def ModelSetUp_1st(data, Constants,Cuts):
    model = ConcreteModel()

    # Define sets and parameters specific to the first stage
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')
    model.solar = Set(initialize=["Solar"])  #Set(initialize=data['solar'].index)  # Set of solar types (e.g., 'Solar')
    model.periods = Set(initialize=L.keys())   # Set of time periods (e.g., 1, 2, ..., 24)
    model.penalty= Set(initialize=['Load_penalty']) 
    model.scenarios=pyo.Set(initialize=Constants['Scenarios'])

    #Parameters
    model.Li=pyo.Param(model.penalty, initialize=Constants['Load_penalty'])
    model.Ci=pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.Si=pyo.Param(model.solar, initialize=data['solar']['Ci'])
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.ki=pyo.Param(model.solar, initialize=data['solar']['yi'])
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.Phi_min=pyo.Param(model.solar, initialize=data['solar']['P_min'])
    model.Phi_max=pyo.Param(model.solar, initialize=data['solar']['P_max'])

    #Maximum hydro production available for the timesteps
    #model.Hmax = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])
    # Load data
    model.L = Param(model.periods, initialize=lambda model, j: L[j])
    
    # Variables (all four are complicating variables as they are present in both stages)
    model.p   = pyo.Var(model.plants, model.periods, bounds=p_bounds)  # Production from hydro plants
    model.H_cap = pyo.Var(model.plants, model.periods, bounds=Hydro_capacity, initialize=Constants['Hydro_cap'])     #Initial capacity - should be a variable as it changes over the time frame due to model.p
    model.phi = pyo.Var(model.solar, model.periods,  within=NonNegativeReals)  # Production from solar installation
    model.L_p = pyo.Var(model.penalty, model.periods, within=NonNegativeReals)  # Load penalty
    #model.cap = pyo.Var(model.plants, within=NonNegativeReals, initialize=Constants['Hydro_cap'])

    # Constraints
    model.load_cons = pyo.Constraint(model.periods, rule=load_rule)                        # 1
    model.hydro_cons=pyo.Constraint(model.plants, model.periods, rule=Hydro_capacity)      # 2
    model.pbounds = pyo.Constraint(model.plants, model.periods, rule = p_bounds)           # 3
    model.solar_constraint = pyo.Constraint(model)                                         # 4
    
    
    
    """Cuts_information"""
    #Set for cuts
    model.Cut = pyo.Set(initialize = Cuts["Set"])
    #Parameter for cuts
    model.Phi = pyo.Param(model.Cut, initialize = Cuts["Phi"])
    model.Lambda = pyo.Param(model.Cut, model.periods, initialize = Cuts["lambda"])
    model.x_hat = pyo.Param(model.Cut, model.periods, initialize = Cuts["x_hat"])
    #Variable for alpha
    model.alpha = pyo.Var(bounds = (-100000000,100000000))
    
    """Constraint cut"""
    model.CreateCuts = pyo.Constraint(model.Cut, rule = CreateCuts)                         # 5 (alpha constraint adressing the complicatin variables)


    """Constraints"""
    # Define objective function
    model.obj = pyo.Objective(rule=Obj_1st, sense=pyo.minimize)
    
    return model



def ModelSetUp_2nd(data, Constants, X_hat):
    # Instance
    model = pyo.ConcreteModel()
    # Define sets
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')
    model.solar = Set(initialize=['S_high','S_avg','S_low'])  # Set of solar types (e.g., 'Solar')
    model.periods = Set(initialize=L.keys()) 
    model.penalty= Set(initialize=['Load_penalty']) 

    # Define parameters
    model.X_hat = pyo.Param(model.plants, model.periods, initialize=X_hat)
    model.Li=pyo.Param(model.penalty, initialize=Constants['Load_penalty'])
    model.Ci=pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.Si=pyo.Param(model.solar, initialize=data['solar']['Ci'])
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.ki=pyo.Param(model.solar, initialize=data['solar']['yi'])
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.Phi_min=pyo.Param(model.solar, initialize=data['solar']['P_min'])
    model.Phi_max=pyo.Param(model.solar, initialize=data['solar']['P_max'])
    

    # Define variables
    model.p     = pyo.Var(model.plants, model.periods, within=p_bounds)
    model.H_cap = pyo.Var(model.plants, model.periods, model.scenarios, within=Hydro_capacity2, initialize=rem_cap)
    model.phi   = pyo.Var(model.solar, model.periods, model.scenarios, within=NonNegativeReals)
    model.L_p   = pyo.Var(model.penalty, model.periods, within=NonNegativeReals)
    

    #Constraints
    model.pbounds = pyo.Constraint(model.plants, rule=p_bounds)
    model.load_cons_TwoStage = pyo.Constraint(model.scenarios, model.periods, rule=load_rule_scenarios)
    model.solar_constraint = pyo.Constraint(rule=Solar_rule)
    model.remaining_capacity = pyo.Constraint(model.plants, rule=rem_cap)                                           #LINKING CONSTRAINT
    model.hydro_cons = pyo.Constraint(model.plants, rule=Hydro_capacity2)                                           #constraining power production from hydrom from the remaining capacity
    
    
    # Define objective function
    model.obj = pyo.Objective(rule=Obj_2nd, sense=pyo.minimize)
    return model


def Solve(model):
    opt = SolverFactory("gurobi")
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    results = opt.solve(model, load_solutions=True)
    return results, model
def DisplayResults(model):
    return print(model.display(), model.dual.display())


## Cutting

In [62]:
# ----------------------- CUTTING ------------------------ #

# Function for creating new linear cuts for optimization problem
def Cut_manage(Cuts,model):
    """Add new cut to existing dictionary of cut information"""
    
    #Find cut iteration by checking number of existing cuts
    cut = len(Cuts["Set"])
    #Add new cut to list, since 0-index is a thing this works well
    Cuts["Set"].append(cut)
    
    #Find 2nd stage cost result
    Cuts["Phi"][cut] = pyo.value(model.obj)
    #Find lambda x_hat for each type of power produced
    for i in model.plants:
        try:
            Cuts["lambda"][cut, i] = model.dual.get(model.H_stage2[i], 0)  # Use a default value if the key doesn't exist
        except KeyError:
            print(f"Key not found for {i}: {model.H_stage2[i]}")
        Cuts["x_hat"][cut,i] = model.X_hat[i]
    return(Cuts)

Cuts = {}
Cuts["Set"] = []
Cuts["Phi"] = {}                #objective value calculated in stage 1
Cuts["lambda"] = {}             #
Cuts["x_hat"] = {}


## Solving model

In [63]:
#This is the while-loop in principle, but for this case is only a for-loop
for i in range(10):

    #Solve 1st stage problem
    m_1st = ModelSetUp_1st(data, Constants,Cuts)
    Solve(m_1st)
 
 
    # Process 1st stage result
    X_hat = {}
    for j in m_1st.periods:
        X_hat_j = {
            "Hydro1": m_1st.p["Hydro1", j].value,
            "Hydro2": m_1st.p["Hydro2", j].value,
            #"Solar":  m_1st.phi["Solar", j].value,                  #are the solar and load penalty values necessary? They are merely slack variables in  way. naaah
            #"Load_penalty": m_1st.L_p['Load_penalty', j].value
        }
        X_hat[j] = X_hat_j
    display(X_hat)

    # Setup and solve 2nd stage problem with updated H_stage1
    m_2nd = ModelSetUp_2nd(data, Constants, X_hat, m_1st.H_stage2)      #must take into account the capacity used in stage 1 as a linking variable
    Solve(m_2nd)

    #Create new cuts for 1st stage problem
    Cuts = Cut_manage(Cuts,m_2nd)
    
    #Print results 2nd stage
    print("Objective function:",pyo.value(m_2nd.obj))
    print("Cut information acquired:")
    for component in Cuts:
        if component == "lambda" or component == "x_hat":
            for j in m_2nd.plants:
                print(component,j,Cuts[component][i,j])
        else:
            print(component,Cuts[component])
    input()
    
    #We perform a convergence check
    print("UB:",pyo.value(m_1st.alpha.value),"- LB:",pyo.value(m_2nd.obj))
    input()

def displayresults(model):
    return print(model.display()) #model.dual.display())

ERROR: Rule failed when initializing variable for Var H_cap with index None:
TypeError: '_GeneralVarData' object is not iterable
ERROR: Constructing component 'H_cap' from data=None failed: TypeError:
'_GeneralVarData' object is not iterable


TypeError: '_GeneralVarData' object is not iterable

In [71]:
import pyomo.environ as pyo
import numpy as np
from pyomo.environ import ConcreteModel,Set,RangeSet,Param,Suffix,Reals,NonNegativeReals,NonPositiveReals,Binary,Objective,minimize,maximize,value
from pyomo.core import Constraint,Var,Block,ConstraintList
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
import matplotlib.pyplot as plt
from calculations.datahandling import*
from calculations.data_processor import* 

#--Data handling--
#Read of parameters for portfolio
def InputData(data_file):
    inputdata = pd.read_excel(data_file)
    inputdata = inputdata.set_index('Parameter', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['hydro'] = inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Solar')
    data['solar']=inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Hydro1').drop('Hydro2')
    return data

data = InputData('data/Parameters.xlsx')


#Solar production based on forecast 
def read_solar_data(irrad_file, spec_file, start_date, end_date):
    input_data_PV = read_excel_data(spec_file)
    input_data_Irr = read_irr_data(irrad_file)
    PV_power = pv_power_estimated(input_data_PV,input_data_Irr)
    Solar_1=convert_to_dict(PV_power, start_date, end_date, 'H')
    Solar_p=scale_dict(Solar_1, 1)
    return Solar_p

#Start and end dates of the optimization
start_date='2018-05-28 00:00'
end_date='2018-05-28 23:00'     #Two time steps only

#Original forecast for solar power production
Solar_p=read_solar_data('data/Data_solar_irr_NOR.csv', 'data/PV_spec.xlsx', start_date, end_date)

S_high=scale_dict(Solar_p, 1.5)
S_avg=scale_dict(Solar_p, 1)
S_low=scale_dict(Solar_p, 0.5)

#Load 
L= {1:50, 3:20, 4:30, 5:50, 6:80, 7:50, 8:90, 9:110, 10:150, 11:120, 12:80, 13:70, 14:80, 15:90, 16:160, 17:170, 18:150, 19:120, 20:100, 21:70, 22:60, 23:50, 24:40} 

#--Constants--
Constants= {
    'Load_penalty':100, 
    'Hydro_cap':3000,
    'Scenarios':['S_high', 'S_avg', 'S_low'], 
    'probs':{'S_high':1/3, 'S_avg':1/3, 'S_low':1/3},
    'yi_s1':{'S_high':25, 'S_avg':25, 'S_low':25}, 
    'yi_s2':{'S_high':35, 'S_avg':35, 'S_low':35}     
}

# ----------------------Mathematical formulation ------------------------- #
# ----- First stage obejctive and constraints ------------------ #
def Obj_1st(model):
    return sum(model.yi[i] * model.p[i, j] for i in model.plants for j in model.periods) \
         + sum(model.ki[s] * model.phi[s, j] for s in model.solar for j in model.periods) \
         + sum(model.Li[n] * model.L_p[n, j] for n in model.penalty for j in model.periods) + model.alpha

def load_rule(model,j):
    return model.p['Hydro1',j] + model.p['Hydro2',j] + model.phi['Solar',j] + model.L_p['Load_penalty',j] == model.L[j]

def Hydro_capacity(model,i,j):
    return (model.H_capmin[i],model.H_capmax[i])
    #return model.p[i,j] <= model.H_cap[i,j]

def hydro_capacity_change_rule(model, i, j): #Just like the volume change rule except now the capacity is steadily decreasing over time as model.p generates power
    if j == 1:
        return model.H_cap[i, j] == Constants['Hydro_cap'] - model.p[i, j]
    else:
        return model.H_cap[i, j] == model.H_cap[i, j - 1] - model.p[i, j]


def p_bounds(model,i,j):
    return (model.Pmin[i],model.Pmax[i])   

def Solar_rule(model,j):
    return model.phi['Solar',j] == Solar_p[j]

def CreateCuts(model,Ci):
    return(model.alpha <= model.Phi[Ci] + sum(model.Lambda[Ci,i]*(model.p[i]-model.x_hat[Ci,i]) for i in model.plants))  #only takes into account the hydro power production scheduling from H_cap

#def CreateCuts(model)
#    return(model.alpha >= model.Phi[])

#def CreateCuts(m,c):
#    return(m.alpha <= m.Phi[c] + sum(m.Lambda[c,i]*(m.x[i]-m.x_hat[c,i]) for i in m.I))  #only the variable x is taken into account as it is the only complicating variable
#the sensitivities are indicated by

# ----- Second stage ojbective and constraints --------------#
def Obj_2nd(model):
    return sum(model.probs[l] * (model.yi[i]*model.p[i,l,j] + model.ki[s] * model.phi[l, j] + model.Li[n] * model.L_p[l, j]) for i in model.plants for s in model.solar for l in model.scenarios for n in model.penalty for j in model.periods)

def load_rule_scenarios(model,s,j,l): #l in scenarios
    return model.p['Hydro1',l,j] + model.p['Hydro2',l,j] + (model.phi[s,l,j]) + model.L_p['Load_penalty'] == model.L[j]

def Hydro_capacity_scenarios(model, i,j,l):
    return (model.H_capmin[i],model.H_capmax[i])

    #return model.p[i,j,l] <= model.H_cap[i,j,l]
def Solar_rule_scenarios(model, j, s, l):
    return model.phi[s, j, l] == {
        'S_high': S_high[j],
        'S_avg': S_avg[j],
        'S_low': S_low[j]
    }[s]

def p_bounds(model,i,j):
    return (model.Pmin[i], model.Pmax[i])   

#def Solar_high(model,j):
#    return model.phi['S_high',j]==S_high[j]

#def Solar_avg(model,j):
#    return  model.phi['S_avg',j] == S_avg[j]

#def Solar_low(model,j):
#    return  model.phi['S_low',j] == S_low[j]

def rem_cap(Constants, X_hat,j):
    remaining_cap = 0
    return remaining_cap.append(Constants['Hydro_cap'] - sum(X_hat['Hydro1',j] + X_hat['Hydro2',j]))

# ------------------------- MODEL SETUP -------------------------- #

# Set up model 1st stage
def ModelSetUp_1st(data, Constants,Cuts):
    model = ConcreteModel()

    # Define sets and parameters specific to the first stage
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')
    model.solar = Set(initialize=["Solar"])  #Set(initialize=data['solar'].index)  # Set of solar types (e.g., 'Solar')
    model.periods = Set(initialize=L.keys())   # Set of time periods (e.g., 1, 2, ..., 24)
    model.penalty= Set(initialize=['Load_penalty']) 
    model.scenarios=pyo.Set(initialize=Constants['Scenarios'])

    #Parameters
    model.Li=pyo.Param(model.penalty, initialize=Constants['Load_penalty'])
    model.Ci=pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.Si=pyo.Param(model.solar, initialize=data['solar']['Ci'])
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.ki=pyo.Param(model.solar, initialize=data['solar']['yi'])
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.Phi_min=pyo.Param(model.solar, initialize=data['solar']['P_min'])
    model.Phi_max=pyo.Param(model.solar, initialize=data['solar']['P_max'])
    model.H_capmax = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])     #Max allowed hydro capacity
    model.H_capmin = pyo.Param(model.plants, initialize=0)                          #Minimum allowed hydro capacity
    model.H0_cap = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])       #Starting hydro capacity
    model.H_cap = pyo.Param(model.plants, model.periods, within=NonNegativeReals, initialize=Constants['Hydro_cap'])

    #Maximum hydro production available for the timesteps
    #model.Hmax = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])
    # Load data
    model.L = Param(model.periods, initialize=lambda model, j: L[j])
    
    # Variables (all four are complicating variables as they are present in both stages)
    model.p   = pyo.Var(model.plants, model.periods, bounds=p_bounds)  # Production from hydro plants
    model.phi = pyo.Var(model.solar, model.periods,  within=NonNegativeReals)  # Production from solar installation
    model.L_p = pyo.Var(model.penalty, model.periods, within=NonNegativeReals)  # Load penalty
    model.H_cap = pyo.Var(model.plants, model.periods, within=NonNegativeReals, initialize=Constants['Hydro_cap'])

    # Constraints
    model.load_cons = pyo.Constraint(model.periods, rule=load_rule)                        # 1
    model.hydro_cons=pyo.Constraint(model.plants, model.periods, rule=Hydro_capacity)      # 2
    model.pbounds = pyo.Constraint(model.plants, model.periods, rule = p_bounds)           # 3
    model.solar_constraint = pyo.Constraint(model.periods, rule=Solar_rule)                # 4
    model.hydro_capacity_change_cons = Constraint(model.plants, model.periods, rule=hydro_capacity_change_rule) 

    
    
    """Cuts_information"""
    #Set for cuts
    model.Cut = pyo.Set(initialize = Cuts["Set"])
    #Parameter for cuts
    model.Phi = pyo.Param(model.Cut, initialize = Cuts["Phi"])
    model.Lambda = pyo.Param(model.Cut, model.periods, initialize = Cuts["lambda"])
    model.x_hat = pyo.Param(model.Cut, model.periods, initialize = Cuts["x_hat"])
    #Variable for alpha
    model.alpha = pyo.Var(bounds = (-100000000,100000000))
    
    """Constraint cut"""
    model.CreateCuts = pyo.Constraint(model.Cut, rule = CreateCuts)                         # 5 (alpha constraint adressing the complicatin variables)


    """Constraints"""
    # Define objective function
    model.obj = pyo.Objective(rule=Obj_1st, sense=pyo.minimize)
    
    return model



def ModelSetUp_2nd(data, Constants, X_hat):
    # Instance
    model = pyo.ConcreteModel()
    # Define sets
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')
    model.solar = Set(initialize=Constants['Scenarios'.index])  # Set of solar types (e.g., 'Solar')
    model.periods = Set(initialize=L.keys()) 
    model.penalty= Set(initialize=['Load_penalty']) 

    # Define parameters
    model.X_hat = pyo.Param(model.plants, model.periods, initialize=X_hat)
    model.Li=pyo.Param(model.penalty, initialize=Constants['Load_penalty'])
    model.Ci=pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.Si=pyo.Param(model.solar, initialize=data['solar']['Ci'])
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.ki=pyo.Param(model.solar, initialize=data['solar']['yi'])
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.Phi_min=pyo.Param(model.solar, initialize=data['solar']['P_min'])
    model.Phi_max=pyo.Param(model.solar, initialize=data['solar']['P_max'])
    model.H_capmax = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])     #Max allowed hydro capacity
    model.H_capmin = pyo.Param(model.plants, initialize=0)                          #Minimum allowed hydro capacity
    model.H0_cap = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])       #Starting hydro capacity for stage 2, should be the rem_cap value from that constraint

    # Define variables
    model.p     = pyo.Var(model.plants, model.periods, within=p_bounds)
    model.H_cap = pyo.Var(model.plants, model.periods, model.scenarios, within=Hydro_capacity_scenarios, initialize=rem_cap)
    model.phi   = pyo.Var(model.solar, model.periods, model.scenarios, within=NonNegativeReals)
    model.L_p   = pyo.Var(model.penalty, model.periods, within=NonNegativeReals)
    
    #Constraints
    model.pbounds = pyo.Constraint(model.plants, rule=p_bounds)
    model.load_cons_TwoStage = pyo.Constraint(model.scenarios, model.periods, rule=load_rule_scenarios)
    model.solar_constraint = pyo.Constraint(model.periods, model.solar, model.scenarios, rule=Solar_rule_scenarios)
    model.remaining_capacity = pyo.Constraint(model.plants, rule=rem_cap)                                                    #LINKING CONSTRAINT
    model.hydro_cons = pyo.Constraint(model.plants, model.periods, model.scenarios, rule=Hydro_capacity_scenarios)                                           #constraining power production from hydrom from the remaining capacity
    
    
    # Define objective function
    model.obj = pyo.Objective(rule=Obj_2nd, sense=pyo.minimize)
    return model


def Solve(model):
    opt = SolverFactory("gurobi")
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    results = opt.solve(model, load_solutions=True)
    return results, model
def DisplayResults(model):
    return print(model.display(), model.dual.display())

# ----------------------- CUTTING ------------------------ #

# Function for creating new linear cuts for optimization problem
def Cut_manage(Cuts,model):
    """Add new cut to existing dictionary of cut information"""
    
    #Find cut iteration by checking number of existing cuts
    cut = len(Cuts["Set"])
    #Add new cut to list, since 0-index is a thing this works well
    Cuts["Set"].append(cut)
    
    #Find 2nd stage cost result
    Cuts["Phi"][cut] = pyo.value(model.obj)
    #Find lambda x_hat for each type of power produced
    for i in model.plants:
        try:
            Cuts["lambda"][cut, i] = model.dual.get(model.H_stage2[i], 0)  # Use a default value if the key doesn't exist
        except KeyError:
            print(f"Key not found for {i}: {model.H_stage2[i]}")
        Cuts["x_hat"][cut,i] = model.X_hat[i]
    return(Cuts)

Cuts = {}
Cuts["Set"] = []
Cuts["Phi"] = {}                #objective value calculated in stage 1
Cuts["lambda"] = {}             #
Cuts["x_hat"] = {}

#This is the while-loop in principle, but for this case is only a for-loop
for i in range(10):

    #Solve 1st stage problem
    m_1st = ModelSetUp_1st(data, Constants,Cuts)
    Solve(m_1st)
 
 
    # Process 1st stage result
    X_hat = {}
    for j in m_1st.periods:
        X_hat_j = {
            "Hydro1": m_1st.p["Hydro1", j].value,
            "Hydro2": m_1st.p["Hydro2", j].value,
            #"Solar":  m_1st.phi["Solar", j].value,                  #are the solar and load penalty values necessary? They are merely slack variables in  way. naaah
            #"Load_penalty": m_1st.L_p['Load_penalty', j].value
        }
        X_hat[j] = X_hat_j
    display(X_hat)

    # Setup and solve 2nd stage problem with updated H_stage1
    m_2nd = ModelSetUp_2nd(data, Constants, X_hat, m_1st.H_stage2)      #must take into account the capacity used in stage 1 as a linking variable
    Solve(m_2nd)

    #Create new cuts for 1st stage problem
    Cuts = Cut_manage(Cuts,m_2nd)
    
    #Print results 2nd stage
    print("Objective function:",pyo.value(m_2nd.obj))
    print("Cut information acquired:")
    for component in Cuts:
        if component == "lambda" or component == "x_hat":
            for j in m_2nd.plants:
                print(component,j,Cuts[component][i,j])
        else:
            print(component,Cuts[component])
    input()
    
    #We perform a convergence check
    print("UB:",pyo.value(m_1st.alpha.value),"- LB:",pyo.value(m_2nd.obj))
    input()

def displayresults(model):
    return print(model.display()) #model.dual.display())

'pyomo.core.base.param.IndexedParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


RuntimeError: Cannot add component 'H_cap_index' (type <class 'pyomo.core.base.set.SetProduct_OrderedSet'>) to block 'unknown': a component by that name (type <class 'pyomo.core.base.set.SetProduct_OrderedSet'>) is already defined.